In [1]:
import pandas as pd
from nltk.corpus import wordnet
import random
import ipywidgets as widgets
from IPython.display import display
from IPython.display import clear_output

In [2]:
def get_word_info(target_word, korean_word):
    examples = [] 
    definitions = []
    synonyms = []
    antonyms = []

    is_syn_target = False
#     print(target_word)
    syn_targets = []
    for syn in wordnet.synsets(target_word):
        for l in syn.lemmas():
#             print("   >",l)
            if target_word == l.name():
#                 print(l.name)
                syn_targets.append(syn)
                is_syn_target = True
                
                if len(syn_targets) > 5:
                    break
    
    for syn in wordnet.synsets(target_word):
        definitions.append(syn.definition())
        if len(syn.examples()) > 0:
            examples.append(syn.examples())
        for l in syn.lemmas():
            if not target_word == l.name():
                
                for comp_syn in syn_targets:
                    sim_score = comp_syn.wup_similarity(syn)
                    if (sim_score is not None) and \
                        (sim_score > 0.9) and \
                        (l.name() not in synonyms) and \
                        (len(synonyms) < 10): 
                        synonyms.append(l.name())    
#                 if syn_target.wup_similarity(syn) is not None and syn_target.wup_similarity(syn) > 0.6:
#                     synonyms.append(l.name())
            if l.antonyms():
                antonyms.append(l.antonyms()[0].name())
#     print("#1.Word :", target_word)
#     print("#2 definitions : ",set(definitions))
#     print("#3 synonyms : ", set(synonyms))
#     print("#4 antonyms : ", set(antonyms))
#     print(examples)
#     print('\r\n')
    
    random.shuffle(synonyms)

    return {
        "word" : target_word,
        "korean" : korean_word,
        "definitions" : list(set(definitions)),
        "synonyms" : list(set(synonyms)),
        "antonyms" : list(set(antonyms)),
        "examples" : examples
        
    }
# for index,row in read_data.iterrows():
#     res= get_word_info(row['eng'], row['kor'])
#     print(res)


In [3]:
csv_file_src = "/Users/house/desktop/IELTS/IELTS_WORDS_summary.tsv"
read_data = pd.read_csv(csv_file_src, sep='\t', header=None, names=['eng','kor'], encoding='UTF-8')

# display(read_data)

# words = read_data[0].values.tolist()
cnt_defi_notfound = 0
cnt_defi_notfound_list = []
word_corpus = []

for index,row in read_data.iterrows():
    res= get_word_info(row['eng'], row['kor'])
    if len(res["definitions"]) == 0:
        cnt_defi_notfound+=1
        cnt_defi_notfound_list.append(row['eng'])
    word_corpus.append(res)
    

print("WORDS LENGTH : ", len(word_corpus))
# cnt_defi_notfound = 0
# cnt_defi_notfound_list = []
# # word_corpus = []
# for w in words:
#     res = get_word_info(w)
# #     pprint(res)
#     if len(res["definitions"]) == 0:
#         cnt_defi_notfound+=1
#         cnt_defi_notfound_list.append(w)
#     word_corpus.append(res)
print("Not found definitions words length : ", cnt_defi_notfound)
print("Not found definitions words : ",cnt_defi_notfound_list)


# pprint(word_corpus)


WORDS LENGTH :  1012
Not found definitions words length :  89
Not found definitions words :  ['beneficient', 'antecendent', 'predescribe', 'mitaken', 'amoral', 'antiwar', 'antivenom', 'restrospect', 'retroact', 'retrocede', 'afterword', 'donwstream', 'imprision', 'sedue', 'corrput', 'heptathon', 'centrigrade', 'polythesim', 'account for', 'at the mercy of', 'attribute A to B', 'be prone to', 'bear in mind', 'behind the times', 'bring about', 'bring home to', 'carry out', 'come up with', 'consist of', 'cut a fine figure', 'devote oneself to', 'distinguish oneself', 'divide into', 'do away with', 'find fault with', 'for the sake of', 'in addition to', 'in behalf of', 'in the presence of', 'in virtue of', 'keep pace with', 'keep track of', 'lag behind', 'make allowances for', 'make good', 'prior to', 'set forth', 'stand for ', 'take into account', 'take it for granted', 'take up', 'turn into', 'up to', 'wear out', 'fauna and flora', 'invertbrate', 'blosoom', 'kindgom', 'evolutionary theor

In [4]:
def q_rand_word(word_corpus):
    
    rand_num = random.randrange(1, len(word_corpus))
    print(word_corpus[rand_num]['word'])
          

In [5]:
def q_multiple_choice(word_corpus,mode='default'):
    
    rand_num = random.randrange(1, len(word_corpus))
    
    if mode == 'default':
        question_field = 'korean'
        target_field = 'word'
    elif mode =='eng':
        question_field = 'word'
        target_field = 'korean'        
    elif mode == "synonyms":
        question_field = 'synonyms'
        target_field = 'word'
        
    target_word = word_corpus[rand_num][target_field]
    question_word = word_corpus[rand_num][question_field]    
    choices = []
    for _ in range(50):
        choice_num = random.randrange(1, len(word_corpus))
        choice_word = word_corpus[choice_num][target_field]

        if (choice_word not in choices) and (target_word is not choice_word):
            choices.append(choice_word)
        if len(choices) >= 3:
            break
    return [question_word, target_word, choices]



"""
    1. 한글 단어 -> 영어 맞추기 / default 
    2. 영어 단어 -> 한글 뜻 맞추기 / eng
    3. 유사어,동의어 -> 영어 맞추기 / synonym
"""

#     display(target_word)
#     display(choices)

'\n    1. 한글 단어 -> 영어 맞추기 / default \n    2. 영어 단어 -> 한글 뜻 맞추기 / eng\n    3. 유사어,동의어 -> 영어 맞추기 / synonym\n'

In [6]:
    
def q_fill_word(word_corpus, mode="default"):
    rand_num = random.randrange(0, len(word_corpus))
    
    question = word_corpus[rand_num]["korean"]
    example = word_corpus[rand_num]["word"]
    answer = word_corpus[rand_num]["word"]
    
    t_nums = []
    for _ in range(10):
        t_rnum = random.randrange(0, len(answer))
        if t_rnum not in t_nums:
            t_nums.append(t_rnum)
        if len(t_nums) == 2:
            break
        
    example=""
    for idx in range(len(answer)):
        if idx in t_nums:
            example+=answer[idx]
        else:
            example+="O"
        
        
        
        
            
    return question, example, answer
    
    
# print(q_fill_word(word_corpus,"default"))
    
    
    

In [7]:
def create_multipleChoice_widget(description, options, correct_answer):
    if correct_answer not in options:
        options.append(correct_answer)
    
    random.shuffle(options)
    correct_answer_index = options.index(correct_answer)
    
    radio_options = [(words, i) for i, words in enumerate(options)]
    alternativ = widgets.RadioButtons(
        options = radio_options,
        description = '',
        disabled = False
    )
    
    description_out = widgets.Output()
    with description_out:
        print(description)
        
    feedback_out = widgets.Output()

    def check_selection(b):
        a = int(alternativ.value)
        if a==correct_answer_index:
            s = '\x1b[5;30;42m' + "True." + '\x1b[0m' +"\n" #green color
        else:
            s = '\x1b[5;30;41m' + "False. " + '\x1b[0m' +"\n" #red color
        with feedback_out:
            clear_output()
            print(s)
        return
    
    check = widgets.Button(description="Check")
    check.on_click(check_selection)
    
    
    return widgets.VBox([description_out, alternativ, check, feedback_out])



In [8]:
def create_textFill_widget(question, hint, answer):
    description_out = widgets.Output()
    with description_out:
        print(question,", ", hint)
    text_fill_out = widgets.Output()    
    text_box = widgets.Text(
        value="",
        placeholder=hint,
        description="",
        disabled=False
    )
    
    
    feedback_out = widgets.Output()
    def check_fill(b):
        answer_value = text_box.value
        if answer_value==answer:
            s = '\x1b[5;30;42m' + "True." + '\x1b[0m' +"\n" #green color
        else:
            s = '\x1b[5;30;41m' + "False. " + '\x1b[0m' +"\n" #red color
        with feedback_out:
            clear_output()
            print(s)
        return
    
    check = widgets.Button(description="Check")
    check.on_click(check_fill)
    
    return widgets.VBox([description_out, text_box, check, feedback_out])


In [9]:
def get_questions_general(num):
    cnt=0
    for _ in range(num):
        question, target_word, example = q_multiple_choice(word_corpus,"default")
#         print(question.join(","))
        cnt+=1
        temp_quest = create_multipleChoice_widget("Q"+str(cnt)+". "+str(question),example ,target_word)
        display(temp_quest)
        
        
def get_questions_eng(num):
    cnt=0
    for _ in range(num):
        question, target_word, example = q_multiple_choice(word_corpus,"eng")
#         print(question.join(","))
        cnt+=1
        temp_quest = create_multipleChoice_widget("Q"+str(cnt)+". "+question,example ,target_word)
        display(temp_quest)

def get_questions_synonyms(num):
    cnt=0
    for _ in range(num*10):
        question, target_word, example = q_multiple_choice(word_corpus,"synonyms")
#         print(question)
        if (question == None) or (len(question)==0): 
            continue
        if cnt == num:
            break
#         print(question.join(","))
        cnt+=1
        temp_quest = create_multipleChoice_widget("Q"+str(cnt)+". "+", ".join(question),example ,target_word)
        display(temp_quest)
        
        
        
def get_questions_filling(num):
    cnt=0
    for _ in range(num*10):
        question,example,answer = q_fill_word(word_corpus,"default")
        if ' ' in question:
            continue
        
        if cnt == num:
            break
        cnt+=1
        temp_quest=create_textFill_widget("Q"+str(cnt)+". " + question,example,answer)
        display(temp_quest)

In [10]:
get_questions_general(100)

In [11]:
get_questions_eng(100)

In [12]:
get_questions_synonyms(100)

In [13]:
# Function add.make function for fill the empty word(have a 2-3 hint character)

get_questions_filling(100)

In [15]:
#Function add.make function using example sentence(fill) -> 보기 4-5개 주기 (문장 안에 채워넣기) 

cnt_not_exists =0

for i in word_corpus:
    if len(i['examples']) == 0:
        cnt_not_exists+=1
#         print(cnt_not_exists,i['word'])
    else :
        print("*", i['word'])
        for j in i['examples']:
            for k in j:
                if i['word'] in k:
                    print(">",k)
        print("---------------")
#         print("* ", i['word'])
#         pprint(i['examples'])

* aeration
> the aeration of the soil
---------------
* aerodynamic
---------------
* aerobic
> aerobic fermentation
> aerobic dance
> running is very aerobic
---------------
* alternate
> the cleaning lady comes on alternate Wednesdays
> stems with alternate leaves
---------------
* ambidextrous
> an ambidextrous surgeon
---------------
* ambiguous
> the polling had a complex and equivocal (or ambiguous) message for potential female candidates
> ambiguous words
> frustrated by ambiguous instructions, the parents were unable to assemble the toy
> an ambiguous situation with no frame of reference
> ambiguous inkblots
---------------
* amphibious
> amphibious vehicles
> amphibious operations
> amphibious troops
> frogs are amphibious animals
---------------
* archetype
---------------
* catastrophe
> lack of funds has resulted in a catastrophe for our school system
---------------
* autograph
> The author autographed his book
---------------
* embargo
> embargoed publications
> The U.S. 

* phonetic
> phonetic transcription
> phonetic analysis
---------------
* syllable
> the word `pocket' has two syllables
---------------
* abridge
> the new law might abridge our freedom of expression
---------------
* satire
---------------
* criticism
> the senator received severe criticism from his opponent
> constructive criticism is always appreciated
---------------
* regard
> give him my kind regards
> a man who has earned high regard
> she mistook his manly regard for love
---------------
* starve
> The political prisoners starved to death
> They starved the prisoners
> The engine was starved of fuel
---------------
* sense
> a sense of security
> a sense of happiness
> a sense of danger
> a sense of self
> the dictionary gave several senses for the word
> in the best sense charity is really a duty
> in the dark he had to depend on touch and on his senses of smell and hearing
> Common sense is not so common
> he hasn't got the sense God gave little green apples
> fortunately sh